In [1]:
import numpy as np

In [2]:
%load_ext Cython

In [3]:
%%cython
cimport cython
cimport numpy as np
import numpy as np
from libc.math cimport log2
from cython.parallel import prange, parallel

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef double entropy(double[:, :] y):
    cdef float n = y.shape[0]
    
    _,  cnts = np.unique(y, return_counts=True)
    if len(cnts) == 1:
        return 0
    cdef double accum = 0;
    cdef int i = 0;
    cdef double p;
    for i in range(len(cnts)):
        p = cnts[i] / n
        accum += p * log2(p)
    return -accum

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef double gini(double[:, :] y):
    cdef float n = y.shape[0]
    
    _,  cnts = np.unique(y, return_counts=True)
    cdef double accum = 0;
    cdef int i = 0;
    cdef double p;
    for i in range(len(cnts)):
        p = cnts[i] / n
        accum += p * p
    return 1 - accum

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef double variance(double[:, :] y):
    return np.var(y)

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef double mad_median(double[:, :] y):
    return np.mean(np.abs(y - np.median(y)))

def mode(a):
    if len(a) == 0:
        return np.nan
    (_, idx, counts) = np.unique(a, return_index=True, return_counts=True)
    index = idx[np.argmax(counts)]
    mode = a[index]
    return mode

In [4]:
import numpy as np
from sklearn.base import BaseEstimator

# import pyximport
# pyximport.install(language_level=3)
# from criterion import *



class Node():
    
    def __init__(self, feature_idx=0, threshold=0, labels=None, left=None,
                 right=None, index_mask=None, agg=None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.labels = labels
        self.left = left
        self.left_labels = None
        self.right = right
        self.right_labels = None
        self.agg = agg
        self.index_mask = index_mask
        self.left_mask = None
        self.right_mask = None
        self.n_samples = 0
        if self.labels is not None:
            self.n_samples = len(self.labels)
            
        self.impurity = -1
        self.predict_left = None
        self.predict_right = None
        
    @property
    def is_leaf(self):
        return self.left is None or self.right is None
    
    def __repr__(self):
        r = f'[{self.feature_idx}]'
        r += f"<={self.threshold}"
        r += f'|n={self.n_samples}'
        r += f'|imp={self.impurity}'
        return r


class DecisionTree(BaseEstimator):
    
    def __init__(self, max_depth=np.inf, min_samples_split=2, min_samples_leaf=1,
                 criterion='entropy', debug=False, random_state=None, agg=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.d_mapper = {'gini': gini, 'entropy': entropy, 'variance': variance, 'mad_median': mad_median}
        self.p_mapper = {'mode': mode, 'mean': np.mean, 'median': np.median}
        self.d_func = self.d_mapper[criterion]
        self.debug = debug
        self.tree = None
        self.random_state = random_state
        self.is_clf = criterion in ['gini', 'entropy']
        if agg is None:
            if self.is_clf:
                agg = 'mode'
            else:
                agg = 'mean'
        self.p_func = self.p_mapper[agg]
        self.n_unique_labels = None
        self.tree_max_depth = 0
        self.best_split_attempts = 0
    
    def get_params(self, *args, **kwargs):
        return super().get_params()
    
    def fit(self, X, y):
        np.random.seed(self.random_state)
        
        if self.tree is not None:
            self.tree = None
        self.labels = y
        self.tree_max_depth = 0
        self.best_split_attempts = 0

        if len(X.shape) < 2:
            X = X.reshape(-1, 1)
        if len(y.shape) < 2:
            y = y.reshape(-1, 1)
        if self.is_clf:
            self.n_unique_labels = len(np.unique(y))
        mask = np.full(len(X), True)
        split_queue = [(mask, None, 0)]

        while True:
            if len(split_queue) == 0:
                break
            split_mask, split_root, depth = split_queue.pop(0)
            if split_mask.sum() < self.min_samples_split or depth >= self.max_depth:
                continue

            self.tree_max_depth = max(depth, self.tree_max_depth)

            best_idx, best_t = self._best_split(X, y, split_mask)
            self.best_split_attempts += 1
            if best_idx is None:
                continue
#             print("Best split:", best_idx, "t=", best_t)
            node = Node(feature_idx=best_idx, threshold=best_t, labels=y[split_mask, :], agg=self.d_func)
            left_mask = (X[:, best_idx] <= best_t) & split_mask
            node.left_labels = y[left_mask, :].astype(np.int64).ravel()
            node.left_predict = self.p_func(y[left_mask])
            right_mask = (X[:, best_idx] > best_t) & split_mask
            node.right_labels = y[right_mask, :].astype(np.int64).ravel()
            node.right_predict = self.p_func(y[right_mask])

            
            split_queue.append((left_mask, node, depth + 1))
            split_queue.append((right_mask, node, depth + 1))
            
            if split_root is None:
                self.tree = node
                continue
            if split_root.feature_idx == node.feature_idx:
                if split_root.threshold < node.threshold:
                    if split_root.right:
                        raise ValueError("WTF, This should not happen")
                    split_root.right = node
                elif split_root.threshold > node.threshold:
                    if split_root.left:
                        raise ValueError("WTF, This should not happen 2")
                    split_root.left = node
                else:
                    raise ValueError("Same feature, same threshold")
                
                continue

            if split_root.left is None:
                split_root.left = node
            elif split_root.right is None:
                split_root.right = node
            else:
                raise ValueError("WTF, split root should have at least one vacant place for the next split")
            


    def _best_split(self, X, y, mask):
        X = X[mask, :]
        y = y[mask, :]
        n_samples, n_features = X.shape
        best_idx, best_d, best_t = None, None, None
#         feature_indices = np.random.permutation(n_features)
        for feature_idx in range(n_features):
            d, t = self._best_feature_split(X[:, feature_idx], y, split_half=True)
            if d is None:
                continue
            if best_d is None or d > best_d:
                best_d = d
                best_idx = feature_idx
                best_t = t
        return best_idx, best_t
    
    def _best_feature_split(self, feature, y, split_half=True):
        thresholds = [-1, 0, 1]
        S0 = self.d_func(y)
        n = len(y)
        best_t, best_d = None, None
        for t in thresholds:
            left_mask = feature <= t
            right_mask = feature > t
#             print(left_mask, right_mask)
            yl = y[left_mask, :]
            nl = len(yl)

            yr = y[right_mask, :]
            nr = len(yr)
            
            if nr == 0 or nl == 0 or left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
                continue
            if nr == 0:
                break
            Sl = self.d_func(yl)
            Sr = self.d_func(yr)

            D = S0 - nl / n * Sl - nr / n * Sr
            if best_d is None or D > best_d:
                best_d = D
                best_t = t
        return best_d, best_t
    
    def predict(self, X):
        y = np.full(len(X), np.nan, dtype=self.labels.dtype)

        mask = np.full(len(X), True)
        cur_depth = 0
        queue = [(self.tree, mask)]
        while True:
            if len(queue) == 0:
                break
            node, mask = queue.pop(0)
#             print("CUR NODE:", node, "LEFT:", node.left, "RIGHT:", node.right, "\nMASK:", mask)
            
            left_mask = (X[:, node.feature_idx] <= node.threshold) & mask
            right_mask = (X[:, node.feature_idx] > node.threshold) & mask
#             print("")
            if not node.left:
                y[left_mask] = node.left_predict
            if not node.right:
                y[right_mask] = node.right_predict
            if node.left:
                queue.append((node.left, left_mask))
            if node.right:
                queue.append((node.right, right_mask))
        return y
        
    def predict_proba(self, X):
        y = np.full((len(X), self.n_unique_labels), np.nan, dtype=self.labels.dtype)

        mask = np.full(len(X), True)
        cur_depth = 0
        queue = [(self.tree, mask)]
        while True:
            if not queue:
                break
            node, mask = queue.pop(0)
            left_mask = (X[:, node.feature_idx] <= node.threshold) & mask
            right_mask = (X[:, node.feature_idx] > node.threshold) & mask
            if not node.left:
                cnts = np.bincount(node.left_labels, minlength=self.n_unique_labels)
                
                y[left_mask, :] = cnts / cnts.sum()
            if not node.right:
                cnts = np.bincount(node.right_labels, minlength=self.n_unique_labels)
                y[right_mask] = cnts / cnts.sum()
            if node.left:
                queue.append((node.left, left_mask))
            if node.right:
                queue.append((node.right, right_mask))
        return y

In [5]:
from preproc import *
import pandas as pd

In [6]:
df = pd.read_csv('resources/train.csv', index_col=0)
test_df = pd.read_csv('resources/test.csv', index_col=0)
sub_df = pd.read_csv('resources/sampleSubmission.csv', index_col=0)

delta = df.iloc[:, 0]
Y = df.iloc[:, 1:401].values
X = df.iloc[:, 401:].values.reshape(-1, 20, 20)

kernel_size = 3
X_i = X[delta == 1]
Y_i = Y[delta == 1]

X_train, Y_train = prepare_data(X_i, Y_i, kernel_size=kernel_size)

In [7]:
X_train = X_train.astype(np.float64)

In [8]:
Y_train = Y_train.astype(np.float64)

In [9]:
Xt, Yt = X_train, Y_train

In [32]:
m = DecisionTree(max_depth=3, random_state=42)
m.fit(Xt, Yt)

In [10]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [36]:
accuracy_score(Yt, m.predict(Xt)), roc_auc_score(Yt, m.predict_proba(Xt)[:, 1])

0.883042004048583

In [39]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(Xt, Yt)
accuracy_score(Yt, dt.predict(Xt)), roc_auc_score(Yt, dt.predict_proba(Xt)[:, 1])

(0.883042004048583, 0.8257417033215595)

In [42]:
%%timeit
dt.fit(Xt, Yt)

8.71 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
m.fit(Xt, Yt)

48.1 s ± 11.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
import pyximport
pyximport.install(language_level=3)
from FastDecisionTree import *

In [12]:
fdt = FastDecisionTree(max_depth=3, random_state=42)

In [13]:
%%timeit
fdt.fit(Xt, Yt)

7.69 s ± 493 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
